In [ ]:
pip install --quiet /kaggle/input/kerasapplications

In [ ]:
pip install --quiet /kaggle/input/efficientnet-git

In [ ]:
#26
import efficientnet.tfkeras as efn

In [ ]:
#27
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
# ignoring warnings
import warnings
warnings.simplefilter("ignore")
import os, cv2, json
from PIL import Image
#from tensorflow.keras.applications import EfficientNetB0
#from tensorflow.keras.applications import EfficientNetB3
#from tensorflow.keras.applications import EfficientNetB4
#from tensorflow.keras.applications import EfficientNetB5
#from tensorflow.keras.applications import EfficientNetB6
#from tensorflow.keras.applications import EfficientNetB7
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout
from keras.models import Model, Sequential

In [ ]:
#28
WORK_DIR = '../input/cassava-leaf-disease-classification'
os.listdir(WORK_DIR)

In [ ]:
#29
print('Train images: %d' %len(os.listdir(os.path.join(WORK_DIR, "train_images"))))

In [ ]:
#30
with open(os.path.join(WORK_DIR, "label_num_to_disease_map.json")) as file:
    print(json.dumps(json.loads(file.read()), indent=4))

In [ ]:
train_labels = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
print(train_labels.head())

In [ ]:
#32
sns.countplot(train_labels.label, edgecolor = 'black',
              palette = reversed(sns.color_palette("viridis", 5)))
plt.show()

In [ ]:
data_normall = pd.read_csv("../input/anomaly005/normall_all.csv")
data_normall

In [ ]:
#35
data_anomaly = pd.read_csv("../input/anomaly005/anomaly_all.csv")
data_anomaly

In [ ]:
from sklearn.model_selection import train_test_split
#split the data into train and test set
train,val25 = train_test_split(data_normall, test_size=0.25, random_state=0 , shuffle=True)
#save the data
train.to_csv('train.csv',index=False)
val25.to_csv('val25.csv',index=False)

In [ ]:
import pandas, sys
import pandas as pd


a = pd.read_csv("../input/anomaly005/anomaly_all.csv")
b = pd.read_csv("./val25.csv")

merged =pd.concat([a,b])

merged.to_csv('./val30.csv', index=False)

In [ ]:
data_test =pd.read_csv("./train.csv")
data_test

In [ ]:
data_val = pd.read_csv("./val30.csv")
data_val

In [ ]:
#36
# Main parameters
BATCH_SIZE = 4
STEPS_PER_EPOCH = len(data_test) / BATCH_SIZE
VALIDATION_STEPS = len(data_val) / BATCH_SIZE
EPOCHS = 10
TARGET_SIZE = 512

In [ ]:
#37
data_test.label = data_test.label.astype('str')

train_datagen = ImageDataGenerator(
                                     preprocessing_function = None,
                                     rotation_range = 45,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1,
                                     featurewise_center = True,
                                     featurewise_std_normalization = True)

train_generator = train_datagen.flow_from_dataframe(data_test,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         #subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,shuffle=True,
                         class_mode = "sparse")

In [ ]:
#38
data_val.label = data_val.label.astype('str')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = train_datagen.flow_from_dataframe(data_val,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         #subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,shuffle=True,
                         class_mode = "sparse")

In [ ]:
def create_model():
    model = models.Sequential()
    model.add(efn.EfficientNetB4(include_top = False, weights = "../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b4_noisy-student_notop.h5",
                             input_shape = (TARGET_SIZE, TARGET_SIZE, 3)))
    model.add(layers.GlobalAveragePooling2D())
    
    #model.add(Dense(2048, activation="relu"))    
    model.add(Dense(1024, activation="relu"))
    model.add(Dense(512, activation="relu"))

    model.add(Dropout(0.25))
    
    model.add(Dense(5, activation="softmax"))
    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["sparse_categorical_accuracy"])
    return model
model = create_model()
model.summary()

In [ ]:
#40
print('Our EfficientNet CNN has %d layers' %len(model.layers))

In [ ]:
#41
model_save = ModelCheckpoint('./EffNetB_best_weights.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [ ]:
#from keras.models import load_model
#model_new = load_model('../input/verygood2/outliersModel_20.h5')

In [ ]:
#42
history = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS,
    #callbacks = [model_save, early_stop, reduce_lr]
)

In [ ]:
#43
ss = pd.read_csv(os.path.join(WORK_DIR, "sample_submission.csv"))
ss
preds = []

for image_id in ss.image_id:
    image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
    image = image.resize((TARGET_SIZE, TARGET_SIZE))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))

ss['label'] = preds
ss
ss.to_csv('submission.csv', index = False)

In [ ]:
#44
ss

In [ ]:
#45
model.save('outliersModel4_12.h5')

In [ ]:
#46
model.save_weights('outliersWeight4_12.h5')